[원본 출처](https://machinelearningmastery.com/handle-missing-data-python/)

# 파이썬으로 누락된 값 처리하기 

현실의 데이터는 종종 누락된 값이 존재합니다. 누락된 값을 처리하는 것은 데이터 전처리하는 과정에서 중요합니다. 머신러닝의 알고리즘이 누락된 값을 받아 들이지 못하기 때문이죠.   

이 글에서는 파이썬으로 데이터 중에 누락된 값을 처리하는 방법을 배워보도록 하겠습니다.  바로 시작해 보죠.  


# 1. 피마 인디언의 당뇨병 데이터셋

먼저, 누락된 값이 포함되어 있는 예제로 **피마 인디언 당뇨병** 데이터를 사용하겠습니다. 피마 인디언 당뇨병 데이터셋은 5년동안 수집한 피마 인디언들의 의료기록입니다. 이진 분류 문제로 총 768명의 8개의 입력 변수와 1개의 출력 변수로 구성되어 있습니다. 구체적인 변수는 아래와 같습니다.

1. 임신 횟수
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mmHg).
4. Triceps skinfold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. BMI 수치
7. Diabetes pedigree function.
8. 나이 (years).
9. 당뇨병 여부 (0 or 1).

샘플 데이터셋을 불러오고 상단 5명의 값을 살펴 보겠습니다.

In [1]:
import pandas as pd # 라이브러리 불러오기
# 예제 데이터 URL
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
df = pd.read_csv(data_url, header=None)
# columns 값을 지정
df.columns =[
    'Number of times pregnant',
    'Plasma glucose concentration',
    'Diastolic blood pressure (mmHg)',
    'Triceps skinfold thickness (mm)',
    '2-Hour serum insulin (mu U/ml)',
    'Body mass index',
    'Diabetes pedigree function',
    'Age',
    'Class variable (0 or 1)']
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure (mmHg),Triceps skinfold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index,Diabetes pedigree function,Age,Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


위 데이터에는 누락된 값들이 있습니다. 구체적으로 말씀드리면 결측치들이 0으로 되어있습니다. 
예를 들어 BMI 수치나 혈압값들은 0이 존재할 수 없기에 명백하게 누락된 값들 입니다. 


# 2. 누락된 값 표시 

현재의 데이터셋에는 결측치가 0으로 표시가 되어있습니다. 우리는 이것을 변경하는 방법을 알아 볼겁니다. 
먼저 `pandas.dataframe`에서 제공하는 `describe()`라는 간편한 요약 통계 기능을 사용해서 데이터셋을 살펴보겠습니다.

In [2]:
df.describe()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure (mmHg),Triceps skinfold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index,Diabetes pedigree function,Age,Class variable (0 or 1)
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


위의 결과에서 최소값이 0인 것들이 보이는데요.  다음의 목록의 데이터들은 0의 값을 가질 수 없습니다.

1. Plasma glucose concentration(혈당수치)
2. Diastolic blood pressure(혈압)
3. Triceps skinfold thickness
4. 2-Hour serum insulin
5. BMI 수치

따라서 누락된 값이 0으로 표시되었다고 말할수 있습니다. 누락된 값이 몇개나 되는지 0의 값을 세어 보겠습니다. 

In [3]:
select = ['Plasma glucose concentration',
    'Diastolic blood pressure (mmHg)',
    'Triceps skinfold thickness (mm)',
    '2-Hour serum insulin (mu U/ml)',
    'Body mass index']
print((df[select] == 0).sum())

Plasma glucose concentration         5
Diastolic blood pressure (mmHg)     35
Triceps skinfold thickness (mm)    227
2-Hour serum insulin (mu U/ml)     374
Body mass index                     11
dtype: int64


혈당수치와 BMI값은 상대적으로 적은 결측값을 가지고 있네요, 하지만 인슐린(2-Hour serum insulin) 측정치는 300개가 넘는 결측값을 가집니다. 

파이썬에서는 보통 결측값을 **NaN**으로 처리하는데요,  그 이유는 `sum`, `count` 와 같은 기능을 사용할때 계산에서 제외하기 위해서 입니다. `Pandas`에서 제공하는 `replace()` 기능을 사용해 우리의 데이터셋의 결측값을 NaN으로 변경하겠습니다. 

In [4]:
import numpy as np
df4 = df.copy()
df4[select] = df4[select].replace(0, np.NaN)
df4.tail()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure (mmHg),Triceps skinfold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index,Diabetes pedigree function,Age,Class variable (0 or 1)
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,NaN,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,NaN,NaN,30.1,0.349,47,1
767,1,93.0,70.0,31.0,NaN,30.4,0.315,23,0


첫번째 행(임신횟수)과 마지막 행(당뇨병여부)는 제외하기 위해서 **select** 리스트를 만들어 변경하였습니다.

# 3. 누락된 값이 있는 열 삭제

누락된 값은 몇몇 머신러닝 알고리즘에서 에러를 유발합니다. 예를 들면 **Linear Discriminant Analysis (LDA)** 알고리즘은 데이터셋에 누락값이 있으면 작동하지 않습니다. 이것을 해결하기 위해 가장 간단한 방법으로 누락된 데이터가 있는 열을 삭제해 해결해보도록 하겠습니다. 역시나 `Pandas`에서는 `dropna()` 기능을 통해 손쉽게 처리 할 수 있습니다.  다음과 같이 입력하세요.

In [5]:
# drop rows with missing values
df4.dropna(inplace=True)
# summarize the number of rows and columns in the dataset
print(df4.shape)

(392, 9)


원래 767개의 열이 있었는데, `dropna()` 기능을 사용하니 392개만 남게 되었습니다.  누락된 값을 제거한 이 데이터셋에 LDA 모델을 사용해보겠습니다. 

In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# split dataset into inputs and outputs
values = df4.values
X = values[:,0:8] # input
y = values[:,8] # output
# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.78582892934


성공적으로 정확도를 구할 수 있습니다만, 결측치가 있는 모든 데이터를 삭제하는것은 너무 극단적이죠. 다른 방법으로 결측치를 대체하는 방법에 대해 살펴보겠습니다.

# 4. 누락된 값 대체하기

누락된 값을 대체하는 방법에는 아래와 같이 다양한 방법이 존재 할 수 있습니다. 

1.  특정 상수 값으로 대체, 예를 들면 0.
2.  무작위 값으로 대체.
3.  행의 평균 혹은 중위값으로 대체.
4.  다른 모델을 통한 예측 값으로 대체.

각각의 방법에 따라 모델의 성능이 달라집니다. 따라서 데이터과학자의 능력이 얼마나 출중한지는 나타내는 척도이기도 합니다.  

## 4.1. Pandas fillna() 사용
우리는 여기서 가장 간단한 방법인 행의 평균값으로 누락된 값을 대체해보겠습니다. `Pandas`에서 제공하는 `fillna()` 기능을 사용합니다. 

In [7]:
df2 = df.copy() # 기존 데이터 복사
# 0 to NaN
df2[select] = df2[select].replace(0, np.NaN)
# fill missing values with mean column values
df2.fillna(df.mean(), inplace=True)
# split dataset into inputs and outputs
values = df2.values
X = values[:,0:8] # input
y = values[:,8] # output
# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.76953125


먼저 데이터 셋의 0값을 NaN으로 바꿔주고, 다시 평균값으로 대체해주었습니다.  그리고 나서 모델의 정확도를 측정해보니 좀 더 좋아 진것을 볼 수 있습니다. 

## 4.2. scikit-learn Imputer() 사용

`scikit-learn` 라이브러리에는 `Imputer()`이라는 전처리 기능을 제공해 누락값을 처리 할 수 있게 해줍니다.  이번에는 이것을 사용해 보겠습니다. 

In [10]:
from sklearn.preprocessing import Imputer
df3 = df.copy()
# 0 to NaN
df3[select] = df3[select].replace(0, np.NaN)
# split dataset into inputs and outputs
values = df3.values
X = values[:,0:8]
y = values[:,8]
# fill missing values with mean column values
imputer = Imputer(missing_values='NaN',strategy='mean') # The imputation strategy
transformed_X = imputer.fit_transform(X)
# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, transformed_X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.766927083333


`imputer()` 기능을 좀 더 알아 보시려면 [링크](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html)를 참고 하세요. 모델의 정확도가 미세하지만 좀 더 좋아 졌습니다. 
이렇게 누락된 값을 대체해 가면서 모델의 정확도를 높이기 위한 시도를 하는것도 중요하지만, 결측치가 데이터가 없다는 것 말고도 다른 의미를 가질 수 있다는것에도 유념해야 합니다.  다음으로는 결측치가 있어도 사용 할 수 있는 알고리즘 모델에 대해서 알아보죠. 

# 5. 누락된 값에도 쓸 수 있는 알고리즘

모든 알고리즘이 결측치가 있다고 사용할 수 없는 것은 아닙니다.  **k-Nearest Neighbors**, **xgboost** 같은 견고한 알고리즘은 결측치가 있어도 사용할 수 있습니다.  안타깝게도 `scikit-learn` 에서는  제공하지 않습니다. 보다 자세한 내용은 다음의 링크를 참고해주세요. 
- [Working with missing data, in Pandas](http://pandas.pydata.org/pandas-docs/stable/missing_data.html)
- [Imputation of missing values, in scikit-learn](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values)

# 마치며,

이 튜토리얼에서 누락된 값을 처리한 몇가지 방법에 대해서 알아 보았습니다. 간단히 말하면 아래와 같습니다.

- 어떻게 결측값을 표시하는가? NaN
- 결측값이 있는 데이터를 어떻게 제거 하는가? `dropna()`
- 결측값을 다른 값으로 대체하는 방법은? `imputer()` 

데이터셋의 결측값을 올바르게 처리하는 것이 바로 실력입니다. 저도 계속 공부하겠습니다.